# **Métricas Extras**

**Contenidos**: 

**1.- Métricas de Verdad**: Se compara la versión revisada de las matrices de conexión con la versión generada por las métricas. Se contabilizan los valores correctamente identificados, los que no deberían haber sido relacionados, los no relacionados, y los valores que fueron cambiados a 0 pese a estar relacionados.

Para realizar esta comparación, se deben especificar los capítulos que se desean analizar. Puede tratarse de un solo capítulo, de un rango ordenado de capítulos (por ejemplo, del Capítulo 1 al Capítulo X), o de una selección aleatoria representada por un arreglo del tipo [C1C1, C2C4, C5C7], que indica respectivamente “curso 1, capítulo 1”, “curso 2, capítulo 4” y “curso 5, capítulo 7”. 

**2.- Keywords relacionadas a qué capítulos**: Dado una Keyword, se entrega los capítulos que están relacionadas con el capitulo que contiene la Keyword. 

**3.- Percentiles del XX%**: Recuperar las keywords dentro de este porcentaje dado un capitulo.

## 0.-Librerías y Cargar Archivos de comparación

In [2]:
# Generales (no clean)
import numpy as np
import pandas as pd
import plotly.express as pxpi
import plotly.graph_objects as go
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
import seaborn as sns

In [5]:
# Aqui puede realizar la comparación entre el archivo que
# corresponde a las relaciones inexistentes (0), debiles (1) y fuertes (2) entre cada capitulo
# y el archivo obtenido bajo umbrales y/o percentiles (obtenerlo de forma manual vía stremlit
# y complete los siguientes parámetros).

# Parametros
umbral = "0.50"
percentil = None
percentil_keyword = None
similitary_method = "cosine" # "cosine", "dot_product" , no disponible: "euclidean"
capitulos_seleccionados = "Todos" # Cambiar de ser necesario

# --------------------- #
#  Lectura de Archivos
# --------------------- #

df_cap_keyword_embedding = pd.read_csv("data/capitulos_keywords_with_embeddings.csv")

# Matriz de "adyacencia" revisada por docentes
#df_matriz_conexiones = pd.read_csv("data/matriz_conexiones.csv")
df_matriz_conexiones = pd.read_excel("data/matriz_conexiones.xlsx")


# Matriz de adyacencia dada por las umbrales y percentiles establecidos
if umbral is not None:
    if percentil is not None:
        df_matriz_umbral = pd.read_csv(f"data/matriz_adyacencia_{similitary_method}_percentil{percentil}_umbral{umbral}_{capitulos_seleccionados}.csv")
    elif percentil_keyword is not None:
        df_matriz_umbral = pd.read_csv(f"data/matriz_adyacencia_keywords_{similitary_method}_percentil{percentil}_umbral{umbral}_{capitulos_seleccionados}.csv")
    else:
        df_matriz_umbral = pd.read_csv(f"data/matriz_adyacencia_{similitary_method}_umbral{umbral}_{capitulos_seleccionados}.csv")

# Todas las similitudes entre las keywords para cada par de capitulos (permite seleccionar las keywords y rescatar las keywords necesarias)
#df_all_keyword_similarities = pd.read_csv("data/precomputed_all_keyword_metrics.csv", compression='gzip')
#df_all_keyword_similarities = pd.read_csv("data/precomputed_all_keyword_metrics.csv")

In [4]:
# Observaciones
#df_matriz_conexiones.head(30)
#df_matriz_umbral.head(30)
#df_all_keyword_similarities.head(30)

## 1.- Métricas de Verdad

In [ ]:
# ---------------------------------------------------------
from datetime import datetime

print("=" * 70)
print("🎯 COMPARADOR: Análisis Individual + Global")
print("=" * 70)

# Validación de dimensiones de matrices
print(f"📊 Dimensiones originales:")
print(f"   Docencia: {df_matriz_conexiones.shape}")
print(f"   Umbrales: {df_matriz_umbral.shape}")

# Obtener los índices originales (números)
indices_originales = df_matriz_conexiones.index.tolist()
print(f"📚 Total de capítulos: {len(indices_originales)}")

# Obtener matrices directamente y asegurar que sean numéricas
matriz_docencia = pd.DataFrame(df_matriz_conexiones.values).apply(pd.to_numeric, errors='coerce').fillna(0).astype(int).values
matriz_umbrales = pd.DataFrame(df_matriz_umbral.values).apply(pd.to_numeric, errors='coerce').fillna(0).astype(int).values

# Asegurar que las matrices sean cuadradas del mismo tamaño
min_filas = min(matriz_docencia.shape[0], matriz_umbrales.shape[0])
min_columnas = min(matriz_docencia.shape[1], matriz_umbrales.shape[1])
tamaño_minimo = min(min_filas, min_columnas)

# Recortar ambas matrices al tamaño mínimo común
matriz_docencia = matriz_docencia[:tamaño_minimo, :tamaño_minimo]
matriz_umbrales = matriz_umbrales[:tamaño_minimo, :tamaño_minimo]

# Completar matrices para asegurar simetría (solo si son cuadradas)
matriz_docencia = np.maximum(matriz_docencia, matriz_docencia.T)
matriz_umbrales = np.maximum(matriz_umbrales, matriz_umbrales.T)

# Crear mapeo de índices a formato C1C2 usando df_cap_keyword_embedding
mapeo_indice_a_formato = {}

# Diccionario más profesional para mapeo de cursos
mapeo_cursos = {
    'Primero Básico': '1',
    'Segundo Básico': '2', 
    'Tercero Básico': '3',
    'Cuarto Básico': '4',
    'Quinto Básico': '5',
    'Sexto Básico': '6'
}

# Obtener los IDs reales de los capítulos del dataframe
ids_reales = df_cap_keyword_embedding['id'].unique().tolist()
ids_reales.sort()


# Actualizar indices_originales para que coincida con el tamaño de las matrices
indices_originales = indices_originales[:tamaño_minimo]

# Asumir que el orden de los índices (0, 1, 2, ...) corresponde al orden de los IDs únicos
for i, idx in enumerate(indices_originales):
    if i < len(ids_reales):
        id_real = ids_reales[i]
        # Buscar este ID en el dataframe para obtener información
        cap_info = df_cap_keyword_embedding[df_cap_keyword_embedding['id'] == id_real].iloc[0]
        
        # Crear formato C{curso}C{numero}
        curso = str(cap_info['curso'])
        numero = str(cap_info['numero'])
        
        # Convertir curso a formato C1, C2, etc. usando el diccionario
        curso_num = mapeo_cursos.get(curso, curso)
        formato_c = f"C{curso_num}C{numero}"
        
        mapeo_indice_a_formato[idx] = {
            'formato_c': formato_c,
            'curso': curso,
            'numero': numero,
            'titulo': cap_info['titulo'],
            'id_real': id_real
        }
    else:
        # Si no hay ID correspondiente, crear formato basado en índice
        formato_c = f"Cap{idx+1}"
        mapeo_indice_a_formato[idx] = {
            'formato_c': formato_c,
            'curso': "Desconocido",
            'numero': "?",
            'titulo': f"Capítulo {idx}",
            'id_real': idx
        }

# CLASE COMPARADORA COMPLETA MEJORADA CON ANÁLISIS GLOBAL
class MatrixComparatorMejorado:
    def __init__(self, matriz_docencia, matriz_umbrales, indices_originales, mapeo_indice_a_formato):
        self.matriz_docencia = matriz_docencia
        self.matriz_umbrales = matriz_umbrales
        self.indices_originales = indices_originales
        self.mapeo_indice_a_formato = mapeo_indice_a_formato
        self.n = len(indices_originales)
        self.formatos_c = [mapeo_indice_a_formato[idx]['formato_c'] for idx in indices_originales]
        
        #print(f"✅ Comparador inicializado con {self.n} capítulos")
        #print(f"   - Matriz docencia: {self.matriz_docencia.shape}, tipo: {self.matriz_docencia.dtype}")
        #print(f"   - Matriz umbrales: {self.matriz_umbrales.shape}, tipo: {self.matriz_umbrales.dtype}")

    # MÉTODOS DE SELECCIÓN (LOS MISMOS QUE ANTES)
    def mostrar_capitulos_disponibles(self):
        """Muestra los capítulos disponibles para selección en formato C1C2"""
        print("\n📚 CAPÍTULOS DISPONIBLES:")
        print("-" * 80)
        for i, idx in enumerate(self.indices_originales):
            info = self.mapeo_indice_a_formato[idx]
            formato_c = info['formato_c']
            titulo = info['titulo']
            print(f"{i+1:2d}. {formato_c:6} - {titulo}")
        print("-" * 80)
        print(f"Total: {len(self.indices_originales)} capítulos\n")
    
    def _seleccionar_capitulo_individual(self):
        """Selecciona un solo capítulo en formato C1C2"""
        self.mostrar_capitulos_disponibles()
        
        while True:
            capitulo = input("\nIngresa el identificador del capítulo (ej: C1C1): ").strip()
            
            if capitulo in self.formatos_c:
                idx_pos = self.formatos_c.index(capitulo)
                idx_original = self.indices_originales[idx_pos]
                info = self.mapeo_indice_a_formato[idx_original]
                print(f"✅ Capítulo seleccionado: {capitulo} - {info['titulo']}")
                return [capitulo]
            else:
                print("❌ Capítulo no encontrado. Verifica el formato.")
                print("💡 Revisa la lista de capítulos disponibles arriba.")
    
    def _seleccionar_rango(self):
        """Selecciona un rango de capítulos en formato C1C2"""
        self.mostrar_capitulos_disponibles()
        
        print("\n📊 SELECCIÓN POR RANGO")
        print("Ejemplo: C1C1 (inicio) hasta C3C2 (fin)")
        
        while True:
            try:
                inicio = input("Ingresa el capítulo de INICIO: ").strip()
                fin = input("Ingresa el capítulo de FIN: ").strip()
                
                if inicio not in self.formatos_c:
                    print(f"❌ Capítulo de inicio '{inicio}' no encontrado.")
                    continue
                if fin not in self.formatos_c:
                    print(f"❌ Capítulo de fin '{fin}' no encontrado.")
                    continue
                
                idx_inicio = self.formatos_c.index(inicio)
                idx_fin = self.formatos_c.index(fin)
                
                if idx_inicio > idx_fin:
                    print("❌ El capítulo de inicio debe estar antes del capítulo de fin.")
                    continue
                
                capitulos_seleccionados = self.formatos_c[idx_inicio:idx_fin + 1]
                info_inicio = self.mapeo_indice_a_formato[self.indices_originales[idx_inicio]]
                info_fin = self.mapeo_indice_a_formato[self.indices_originales[idx_fin]]
                
                print(f"✅ Rango seleccionado: {len(capitulos_seleccionados)} capítulos")
                print(f"   Desde: {inicio} - {info_inicio['titulo']}")
                print(f"   Hasta: {fin} - {info_fin['titulo']}")
                
                return capitulos_seleccionados
                
            except KeyboardInterrupt:
                print("\n👋 Operación cancelada.")
                return None
    
    def _seleccionar_lista(self):
        """Selecciona una lista específica de capítulos en formato C1C2"""
        self.mostrar_capitulos_disponibles()
        
        print("\n📝 SELECCIÓN POR LISTA")
        print("Ingresa los capítulos separados por comas")
        print("Ejemplo: C1C1, C2C4, C5C7")
        
        while True:
            try:
                entrada = input("Ingresa los capítulos: ").strip()
                capitulos = [cap.strip() for cap in entrada.split(',')]
                
                capitulos_validos = []
                capitulos_no_encontrados = []
                
                for cap in capitulos:
                    if cap in self.formatos_c:
                        capitulos_validos.append(cap)
                    else:
                        capitulos_no_encontrados.append(cap)
                
                if capitulos_no_encontrados:
                    print(f"❌ Capítulos no encontrados: {capitulos_no_encontrados}")
                    continue
                
                if not capitulos_validos:
                    print("❌ No se seleccionaron capítulos válidos.")
                    continue
                
                print(f"✅ Lista seleccionada: {len(capitulos_validos)} capítulos")
                for cap in capitulos_validos:
                    idx_pos = self.formatos_c.index(cap)
                    idx_original = self.indices_originales[idx_pos]
                    info = self.mapeo_indice_a_formato[idx_original]
                    print(f"   - {cap}: {info['titulo']}")
                
                return capitulos_validos
                
            except KeyboardInterrupt:
                print("\n👋 Operación cancelada.")
                return None
    
    def _seleccionar_todos(self):
        """Selecciona todos los capítulos"""
        print(f"✅ Seleccionados todos los {len(self.formatos_c)} capítulos")
        return self.formatos_c.copy()

    # MÉTODOS DE COMPARACIÓN Y ANÁLISIS
    def comparar_capitulo_vs_todos(self, capitulo_seleccionado):
        """
        Compara un capítulo específico contra TODOS los demás capítulos
        """
        idx_capitulo = self.formatos_c.index(capitulo_seleccionado)
        
        metricas = {
            'correctamente_identificados_total': 0,
            'correctamente_identificados_relacion_fuerte': 0,
            'correctamente_identificados_relacion_debil': 0,
            'incorrectamente_relacionados': 0,
            'no_relacionados': 0,
            'cambiados_a_cero_total': 0,
            'cambiados_a_cero_relacion_fuerte': 0,
            'cambiados_a_cero_relacion_debil': 0,
        }

        resultados = []
        total_comparaciones = 0

        print(f"🔍 Comparando {capitulo_seleccionado} vs TODOS los demás capítulos...")

        for j in range(self.n):
            if j == idx_capitulo:
                continue

            val_doc = self.matriz_docencia[idx_capitulo, j]
            val_umb = self.matriz_umbrales[idx_capitulo, j]

            # Clasificación
            if val_umb == 1 and val_doc in [1, 2]:
                categoria = "Correctamente identificado"
                codigo = "CI"
                metricas['correctamente_identificados_total'] += 1
                if val_doc == 2:
                    categoria = "Correctamente identificado (Relación Fuerte)"
                    codigo = "CI-F"
                    metricas['correctamente_identificados_relacion_fuerte'] += 1
                elif val_doc == 1:
                    categoria = "Correctamente identificado (Relación Débil)"
                    codigo = "CI-D"
                    metricas['correctamente_identificados_relacion_debil'] += 1

            elif val_umb == 1 and val_doc == 0:
                categoria = "Incorrectamente relacionado"
                codigo = "IR"
                metricas['incorrectamente_relacionados'] += 1

            elif val_umb == 0 and val_doc == 0:
                categoria = "No relacionado"
                codigo = "NR"
                metricas['no_relacionados'] += 1

            elif val_umb == 0 and val_doc in [1, 2]:
                categoria = "Cambiado a 0 (relación válida perdida)"
                codigo = "C0"
                metricas['cambiados_a_cero_total'] += 1
                if val_doc == 2:
                    categoria = "Cambiado a 0 (Relación Fuerte perdida)"
                    codigo = "C0-F"
                    metricas['cambiados_a_cero_relacion_fuerte'] += 1
                elif val_doc == 1:
                    categoria = "Cambiado a 0 (Relación Débil perdida)"
                    codigo = "C0-D"
                    metricas['cambiados_a_cero_relacion_debil'] += 1

            else:
                categoria = f"Caso no especificado (docencia={val_doc}, umbrales={val_umb})"
                codigo = "NS"

            resultados.append({
                'Capítulo_seleccionado': capitulo_seleccionado,
                'Capítulo_comparado': self.formatos_c[j],
                'Docencia': val_doc,
                'Umbrales': val_umb,
                'Categoría': categoria,
                'Código': codigo
            })
            total_comparaciones += 1

        df_resultados = pd.DataFrame(resultados)
        
        print(f"✅ Total comparaciones: {total_comparaciones}")

        # Resumen detallado
        print(f"\n📊 RESUMEN DE {capitulo_seleccionado} vs TODOS LOS DEMÁS:")
        print(f"  Correctamente identificados total: {metricas['correctamente_identificados_total']}")
        print(f"     ├─ Coincidencias 1–2 (fuerte): {metricas['correctamente_identificados_relacion_fuerte']}")
        print(f"     └─ Coincidencias 1–1 (débil):  {metricas['correctamente_identificados_relacion_debil']}")
        print(f"  Incorrectamente relacionados (1-0):     {metricas['incorrectamente_relacionados']}")
        print(f"  Relaciones no captadas:              {metricas['cambiados_a_cero_total']}")
        print(f"      ├─ Discrepancia a 0–2 (fuerte):  {metricas['cambiados_a_cero_relacion_fuerte']}")
        print(f"      └─ Discrepancia a 0–1 (débil):   {metricas['cambiados_a_cero_relacion_debil']}")
        print(f"  No relacionados (0–0):            {metricas['no_relacionados']}")

        return df_resultados, metricas, total_comparaciones

    def analizar_multiple_capitulos(self, capitulos_seleccionados):
        """
        Analiza múltiples capítulos y genera estadísticas globales
        """
        print(f"\n🔍 INICIANDO ANÁLISIS GLOBAL DE {len(capitulos_seleccionados)} CAPÍTULOS")
        print("=" * 60)
        
        # Métricas globales acumuladas
        metricas_globales = {
            'correctamente_identificados_total': 0,
            'correctamente_identificados_relacion_fuerte': 0,
            'correctamente_identificados_relacion_debil': 0,
            'incorrectamente_relacionados': 0,
            'no_relacionados': 0,
            'cambiados_a_cero_total': 0,
            'cambiados_a_cero_relacion_fuerte': 0,
            'cambiados_a_cero_relacion_debil': 0,
            'total_comparaciones_global': 0,
            'capitulos_analizados': len(capitulos_seleccionados)
        }
        
        resultados_globales = []
        analisis_individual = {}
        
        # Analizar cada capítulo individualmente
        for i, capitulo in enumerate(capitulos_seleccionados, 1):
            print(f"\n📊 Analizando capítulo {i}/{len(capitulos_seleccionados)}: {capitulo}")
            
            df_resultados, metricas, total_comparaciones = self.comparar_capitulo_vs_todos(capitulo)
            
            # Acumular métricas globales
            for key in metricas_globales:
                if key in metricas:
                    metricas_globales[key] += metricas[key]
            
            metricas_globales['total_comparaciones_global'] += total_comparaciones
            
            # Guardar análisis individual
            analisis_individual[capitulo] = {
                'metricas': metricas,
                'total_comparaciones': total_comparaciones,
                'dataframe': df_resultados
            }
            
            # Acumular resultados para archivo global
            resultados_globales.append(df_resultados)
        
        # Consolidar todos los resultados
        df_global = pd.concat(resultados_globales, ignore_index=True)
        
        print(f"\n✅ ANÁLISIS INDIVIDUAL COMPLETADO")
        print(f"   - Capítulos analizados: {len(capitulos_seleccionados)}")
        print(f"   - Comparaciones totales: {metricas_globales['total_comparaciones_global']}")
        
        return df_global, metricas_globales, analisis_individual

    def generar_estadisticas_globales(self, metricas_globales):
        """
        Genera estadísticas globales consolidadas
        """
        total = metricas_globales['total_comparaciones_global']
        if total == 0:
            print("⚠️ No hay comparaciones globales registradas.")
            return pd.DataFrame()

        def pct(x):
            return round((x / total) * 100, 2)

        data = [
            ["Correctamente identificados (total)",
            metricas_globales["correctamente_identificados_total"],
            pct(metricas_globales["correctamente_identificados_total"])],
            ["   ├─ Relación fuerte (1–2)",
            metricas_globales["correctamente_identificados_relacion_fuerte"],
            pct(metricas_globales["correctamente_identificados_relacion_fuerte"])],
            ["   └─ Relación débil (1–1)",
            metricas_globales["correctamente_identificados_relacion_debil"],
            pct(metricas_globales["correctamente_identificados_relacion_debil"])],

            ["Incorrectamente relacionados (1–0)",
            metricas_globales["incorrectamente_relacionados"],
            pct(metricas_globales["incorrectamente_relacionados"])],

            ["Cambiados a 0 (total)",
            metricas_globales["cambiados_a_cero_total"],
            pct(metricas_globales["cambiados_a_cero_total"])],
            ["   ├─ Fuerte (0–2)",
            metricas_globales["cambiados_a_cero_relacion_fuerte"],
            pct(metricas_globales["cambiados_a_cero_relacion_fuerte"])],
            ["   └─ Débil (0–1)",
            metricas_globales["cambiados_a_cero_relacion_debil"],
            pct(metricas_globales["cambiados_a_cero_relacion_debil"])],

            ["No relacionados (0–0)",
            metricas_globales["no_relacionados"],
            pct(metricas_globales["no_relacionados"])]
        ]

        df_estadisticas = pd.DataFrame(data, columns=["Categoría", "Cantidad", "Porcentaje (%)"])
        return df_estadisticas

    def mostrar_analisis_global(self, metricas_globales, capitulos_seleccionados):
        """Muestra análisis global detallado"""
        total = metricas_globales['total_comparaciones_global']
        if total == 0:
            return
        
        print("\n" + "🌍 ANÁLISIS GLOBAL CONSOLIDADO")
        print("=" * 50)
        
        # Métricas generales
        precision_global = ((metricas_globales['correctamente_identificados_total'] + 
                           metricas_globales['no_relacionados']) / total * 100)
        
        print(f"📈 MÉTRICAS GLOBALES:")
        print(f"   - Capítulos analizados: {len(capitulos_seleccionados)}")
        print(f"   - Total comparaciones: {total}")
        print(f"   - Precisión global: {precision_global:.1f}%")
        print(f"   - Relaciones correctas: {metricas_globales['correctamente_identificados_total']}")
        print(f"   - Falsos positivos: {metricas_globales['incorrectamente_relacionados']}")
        print(f"   - Falsos negativos: {metricas_globales['cambiados_a_cero_total']}")
        print(f"   - No relaciones correctas: {metricas_globales['no_relacionados']}")
        
        # Análisis por tipo de relación
        total_fuertes = (metricas_globales['correctamente_identificados_relacion_fuerte'] + 
                        metricas_globales['cambiados_a_cero_relacion_fuerte'])
        total_debiles = (metricas_globales['correctamente_identificados_relacion_debil'] + 
                        metricas_globales['cambiados_a_cero_relacion_debil'])
        
        if total_fuertes > 0:
            deteccion_fuertes = (metricas_globales['correctamente_identificados_relacion_fuerte'] / 
                               total_fuertes * 100)
            print(f"\n📊 DETECCIÓN RELACIONES FUERTES: {deteccion_fuertes:.1f}%")
        
        if total_debiles > 0:
            deteccion_debiles = (metricas_globales['correctamente_identificados_relacion_debil'] / 
                               total_debiles * 100)
            print(f"📊 DETECCIÓN RELACIONES DÉBILES: {deteccion_debiles:.1f}%")

    def guardar_resultados_globales(self, df_global, metricas_globales, analisis_individual, nombre_archivo, capitulos_seleccionados):
        """Guarda resultados globales consolidados"""
        try:
            nombre_completo = f"{nombre_archivo}_GLOBAL"
            
            # 1. Archivo detallado global (todas las comparaciones)
            df_global.to_csv(f"{nombre_completo}_detallado.csv", index=False, encoding='utf-8-sig')
            
            # 2. Archivo de métricas globales
            metricas_completas = metricas_globales.copy()
            metricas_completas['fecha_analisis'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            metricas_completas['capitulos_seleccionados'] = ', '.join(capitulos_seleccionados)
            
            # Calcular métricas adicionales
            total = metricas_globales['total_comparaciones_global']
            if total > 0:
                metricas_completas['precision_global'] = round(((metricas_globales['correctamente_identificados_total'] + 
                                                              metricas_globales['no_relacionados']) / total * 100), 2)
                metricas_completas['tasa_falsos_positivos'] = round((metricas_globales['incorrectamente_relacionados'] / total * 100), 2)
                metricas_completas['tasa_falsos_negativos'] = round((metricas_globales['cambiados_a_cero_total'] / total * 100), 2)
            
            df_metricas_global = pd.DataFrame([metricas_completas])
            df_metricas_global.to_csv(f"{nombre_completo}_metricas.csv", index=False, encoding='utf-8-sig')
            
            # 3. Archivo resumen global
            with open(f"{nombre_completo}_resumen.txt", 'w', encoding='utf-8') as f:
                f.write("=" * 70 + "\n")
                f.write("RESUMEN GLOBAL DE ANÁLISIS - MÚLTIPLES CAPÍTULOS\n")
                f.write("=" * 70 + "\n\n")
                
                f.write("📋 INFORMACIÓN DEL ANÁLISIS GLOBAL\n")
                f.write("-" * 40 + "\n")
                f.write(f"Fecha de análisis: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
                f.write(f"Capítulos analizados: {len(capitulos_seleccionados)}\n")
                f.write(f"Total comparaciones: {total}\n")
                f.write(f"Capítulos: {', '.join(capitulos_seleccionados)}\n\n")
                
                if total > 0:
                    # Estadísticas globales
                    f.write("📊 ESTADÍSTICAS GLOBALES CONSOLIDADAS\n")
                    f.write("-" * 40 + "\n")
                    
                    estadisticas_global = self.generar_estadisticas_globales(metricas_globales)
                    for _, row in estadisticas_global.iterrows():
                        f.write(f"{row['Categoría']:<45} {row['Cantidad']:>6} ({row['Porcentaje (%)']:>6}%)\n")
                    
                    f.write("\n")
                    
                    # Resumen ejecutivo global
                    f.write("🎯 RESUMEN EJECUTIVO GLOBAL\n")
                    f.write("-" * 40 + "\n")
                    precision_global = ((metricas_globales['correctamente_identificados_total'] + 
                                      metricas_globales['no_relacionados']) / total * 100)
                    f.write(f"Precisión global del modelo: {precision_global:.1f}%\n")
                    f.write(f"Relaciones correctamente identificadas: {metricas_globales['correctamente_identificados_total']}\n")
                    f.write(f"Falsos positivos: {metricas_globales['incorrectamente_relacionados']}\n")
                    f.write(f"Falsos negativos: {metricas_globales['cambiados_a_cero_total']}\n")
                    f.write(f"Aciertos en no-relaciones: {metricas_globales['no_relacionados']}\n")
                
            print(f"\n💾 RESULTADOS GLOBALES GUARDADOS:")
            print(f"   📄 {nombre_completo}_detallado.csv (todas las comparaciones)")
            print(f"   📊 {nombre_completo}_metricas.csv (métricas consolidadas)")
            print(f"   📝 {nombre_completo}_resumen.txt (análisis global completo)")
            
        except Exception as e:
            print(f"❌ Error guardando archivos globales: {e}")

# Crear comparador
comparador = MatrixComparatorMejorado(matriz_docencia, matriz_umbrales, indices_originales, mapeo_indice_a_formato)

# 🎯 SELECCIÓN INTERACTIVA MEJORADA
print(f"\n📋 FORMATOS DE SELECCIÓN:")
print("1. 📍 Un solo capítulo (vs todos los demás)")
print("2. 📊 Rango ordenado de capítulos (análisis individual + global)")
print("3. 🎲 Lista específica de capítulos (análisis individual + global)")
print("4. 🌟 Todos los capítulos (análisis individual + global)")

while True:
    try:
        opcion = input("\nSelecciona el formato (1-4): ").strip()
        
        if opcion == "1":
            capitulos = comparador._seleccionar_capitulo_individual()
            analisis_global = False
            break
        elif opcion == "2":
            capitulos = comparador._seleccionar_rango()
            analisis_global = True
            break
        elif opcion == "3":
            capitulos = comparador._seleccionar_lista()
            analisis_global = True
            break
        elif opcion == "4":
            capitulos = comparador._seleccionar_todos()
            analisis_global = True
            break
        else:
            print("❌ Opción no válida. Por favor, selecciona 1-4.")
    except KeyboardInterrupt:
        print("\n👋 Operación cancelada por el usuario.")
        capitulos = None
        break
    except Exception as e:
        print(f"❌ Error: {e}")

if capitulos is None:
    print("👋 Saliendo del programa.")
else:
    if len(capitulos) == 1 or not analisis_global:
        # ANÁLISIS INDIVIDUAL SIMPLE (como antes)
        for capitulo in capitulos:
            print(f"\n" + "=" * 70)
            print(f"🎯 ANALIZANDO: {capitulo} vs TODOS LOS DEMÁS CAPÍTULOS")
            print("=" * 70)
            
            try:
                df_resultados, metricas, total_comparaciones = comparador.comparar_capitulo_vs_todos(capitulo)
                
                # MOSTRAR RESULTADOS
                print("\n" + "=" * 70)
                print(f"📈 RESULTADOS PARA {capitulo}")
                print("=" * 70)
                
                if total_comparaciones > 0:
                    # Generar estadísticas
                    def pct(x):
                        return round((x / total_comparaciones) * 100, 2)

                    data = [
                        ["Correctamente identificados (total)", metricas["correctamente_identificados_total"], pct(metricas["correctamente_identificados_total"])],
                        ["   ├─ Relación fuerte (1–2)", metricas["correctamente_identificados_relacion_fuerte"], pct(metricas["correctamente_identificados_relacion_fuerte"])],
                        ["   └─ Relación débil (1–1)", metricas["correctamente_identificados_relacion_debil"], pct(metricas["correctamente_identificados_relacion_debil"])],
                        ["Incorrectamente relacionados (1–0)", metricas["incorrectamente_relacionados"], pct(metricas["incorrectamente_relacionados"])],
                        ["Cambiados a 0 (total)", metricas["cambiados_a_cero_total"], pct(metricas["cambiados_a_cero_total"])],
                        ["   ├─ Fuerte (0–2)", metricas["cambiados_a_cero_relacion_fuerte"], pct(metricas["cambiados_a_cero_relacion_fuerte"])],
                        ["   └─ Débil (0–1)", metricas["cambiados_a_cero_relacion_debil"], pct(metricas["cambiados_a_cero_relacion_debil"])],
                        ["No relacionados (0–0)", metricas["no_relacionados"], pct(metricas["no_relacionados"])]
                    ]

                    df_estadisticas = pd.DataFrame(data, columns=["Categoría", "Cantidad", "Porcentaje (%)"])
                    print("\n📈 RESUMEN ESTADÍSTICO:")
                    print(df_estadisticas.to_string(index=False))
                    
                    # Mostrar preview
                    if len(df_resultados) > 0:
                        print(f"\n🔍 PRIMERAS 10 COMPARACIONES:")
                        print(df_resultados.head(10).to_string(index=False))
                else:
                    print("⚠️ No se encontraron relaciones para este capítulo.")
                
                # GUARDAR RESULTADOS EN CSV
                guardar = input(f"\n💾 ¿Guardar resultados para {capitulo} en CSV? (s/n): ").strip().lower()
                if guardar == 's':
                    nombre_base = input("Nombre base para archivos (sin extensión): ").strip()
                    if not nombre_base:
                        nombre_base = f"comparacion_{capitulo}"
                    
                    # Guardado simple (puedes implementar esta función si la necesitas)
                    df_resultados.to_csv(f"{nombre_base}_detallado.csv", index=False, encoding='utf-8-sig')
                    print(f"💾 Resultados guardados en: {nombre_base}_detallado.csv")
                
                print(f"\n✅ Análisis de {capitulo} completado exitosamente!")
                
            except Exception as e:
                print(f"❌ Error durante el análisis de {capitulo}: {e}")

    else:
        # ANÁLISIS MÚLTIPLE CON ESTADÍSTICAS GLOBALES
        print(f"\n" + "=" * 70)
        print(f"🌍 ANALIZANDO {len(capitulos)} CAPÍTULOS (INDIVIDUAL + GLOBAL)")
        print("=" * 70)
        
        # Realizar análisis global
        df_global, metricas_globales, analisis_individual = comparador.analizar_multiple_capitulos(capitulos)
        
        # Mostrar resultados globales
        print("\n" + "=" * 70)
        print("📈 RESULTADOS GLOBALES CONSOLIDADOS")
        print("=" * 70)
        
        stats_global = comparador.generar_estadisticas_globales(metricas_globales)
        print(stats_global.to_string(index=False))
        
        comparador.mostrar_analisis_global(metricas_globales, capitulos)
        
        # Mostrar preview de resultados globales
        if len(df_global) > 0:
            print(f"\n🔍 PRIMERAS 15 COMPARACIONES GLOBALES:")
            print(df_global.head(15).to_string(index=False))
        
        # Guardar resultados globales
        guardar_global = input(f"\n💾 ¿Guardar resultados GLOBALES en CSV? (s/n): ").strip().lower()
        if guardar_global == 's':
            nombre_base = input("Nombre base para archivos globales (sin extensión): ").strip()
            if not nombre_base:
                nombre_base = f"comparacion_global_{len(capitulos)}_capitulos"
            
            comparador.guardar_resultados_globales(df_global, metricas_globales, analisis_individual, nombre_base, capitulos)
        
        print(f"\n🎉 Análisis global de {len(capitulos)} capítulos completado!")

    print(f"\n🏁 Todos los análisis han finalizado!")

### Lectura de metricas totales

In [ ]:
# Lectura resultado SOLO
nombre_adicional_solo = "soloC1C1prueba1"
capitulo_solo = "C1C1"
tipo_lectura_solo = "metricas" # "detallado" o "metricas"
df_resultados_solo = pd.read_csv(f"{nombre_adicional_solo}_{capitulo_solo}_{tipo_lectura_solo}.csv")
display(df_resultados_solo)

# Lectura resultado CONSECUTIVOS
nombre_adicional_conse = "consecutivosC4C1_C4C4prueba2"
tipo_lectura_conse = "metricas" # "detallado" o "metricas"
df_resultados_conse = pd.read_csv(f"{nombre_adicional_conse}_GLOBAL_{tipo_lectura_conse}.csv")
#display(df_resultados_conse)

# Lectura resultado VARIOS
nombre_adicional_varios = "variosC1C1_C4C1_C4C4prueba1"
tipo_lectura_varios = "metricas" # "detallado" o "metricas"
df_resultados_varios = pd.read_csv(f"{nombre_adicional_varios}_GLOBAL_{tipo_lectura_varios}.csv")
#display(df_resultados_varios)

soloC1C1prueba1


,correctamente_identificados_total,correctamente_identificados_relacion_fuerte,correctamente_identificados_relacion_debil,incorrectamente_relacionados,no_relacionados,cambiados_a_cero_total,cambiados_a_cero_relacion_fuerte,cambiados_a_cero_relacion_debil,capitulo_analizado,total_comparaciones,fecha_analisis,precision_general,tasa_falsos_positivos,tasa_falsos_negativos
0,13,13,0,4,71,10,5,5,C1C1,98,2025-11-03 22:15:56,85.71,4.08,10.2


consecutivosC4C1_C4C4prueba2


,correctamente_identificados_total,correctamente_identificados_relacion_fuerte,correctamente_identificados_relacion_debil,incorrectamente_relacionados,no_relacionados,cambiados_a_cero_total,cambiados_a_cero_relacion_fuerte,cambiados_a_cero_relacion_debil,total_comparaciones_global,capitulos_analizados,fecha_analisis,capitulos_seleccionados,precision_global,tasa_falsos_positivos,tasa_falsos_negativos
0,41,37,4,32,246,73,49,24,392,4,2025-11-03 22:47:20,"C4C1, C4C2, C4C3, C4C4",73.21,8.16,18.62


variosC1C1_C4C1_C4C4prueba1


,correctamente_identificados_total,correctamente_identificados_relacion_fuerte,correctamente_identificados_relacion_debil,incorrectamente_relacionados,no_relacionados,cambiados_a_cero_total,cambiados_a_cero_relacion_fuerte,cambiados_a_cero_relacion_debil,total_comparaciones_global,capitulos_analizados,fecha_analisis,capitulos_seleccionados,precision_global,tasa_falsos_positivos,tasa_falsos_negativos
0,22,20,2,16,206,50,34,16,294,3,2025-11-03 22:50:15,"C1C1, C4C1, C4C4",77.55,5.44,17.01


## 2.- Capitulos dado una Keyword

In [ ]:
# --------------------- #
# Aplicación 2: Keywords relacionadas a qué capítulos
# --------------------- #

def aplicacion_2_keywords_capitulos(keyword_buscada, df_cap_keyword_embedding, modo_busqueda="la_contiene", guardar_csv=False, top_n=10):
    """
    Aplicación 2: Encuentra capítulos relacionados con una keyword específica
    
    Parámetros:
    - keyword_buscada: string con la keyword a buscar
    - df_cap_keyword_embedding: DataFrame con las keywords y embeddings
    - modo_busqueda: "exacta" o "la_contiene"
    - guardar_csv: True para guardar resultados en CSV
    - top_n: número máximo de resultados a mostrar
    """
    
    print("=" * 70)
    print("APLICACIÓN 2: Keywords relacionadas a qué capítulos")
    print(f"MODO: {modo_busqueda.upper()}")
    print("=" * 70)
    
    # Verificar que el DataFrame tiene las columnas necesarias
    columnas_requeridas = ['curso', 'numero', 'keywords']
    for col in columnas_requeridas:
        if col not in df_cap_keyword_embedding.columns:
            print(f"Error: Columna '{col}' no encontrada en el DataFrame")
            return None
    
    # Aplicar filtro según el modo de búsqueda
    keyword_buscada_lower = keyword_buscada.lower().strip()
    
    if modo_busqueda == "exacta":
        # BÚSQUEDA EXACTA - busca la frase completa e idéntica
        def contiene_exacta(texto):
            if pd.isna(texto):
                return False
            texto_lower = str(texto).lower().strip()
            # Buscar la frase completa e idéntica
            return keyword_buscada_lower == texto_lower
        
        mascara = df_cap_keyword_embedding['keywords'].apply(contiene_exacta)
        tipo_busqueda = "exacta"
        
    else:  # modo_busqueda == "la_contiene"
        # Búsqueda parcial - la keyword puede estar contenida en cualquier parte
        mascara = df_cap_keyword_embedding['keywords'].str.lower().str.contains(
            keyword_buscada_lower, na=False
        )
        tipo_busqueda = "la_contiene"
    
    coincidencias = df_cap_keyword_embedding[mascara]
    
    if len(coincidencias) == 0:
        print(f"No se encontraron keywords con búsqueda {tipo_busqueda} para: '{keyword_buscada}'")
        
        # Mostrar sugerencias para búsqueda exacta
        if modo_busqueda == "exacta":
            print(f"\n💡 Sugerencia: La búsqueda exacta requiere que la keyword sea IDÉNTICA.")
            print(f"   Prueba con 'la_contiene' o 'exacta flexible'")
            
            # Mostrar algunas keywords similares para ayudar
            print(f"\n🔍 Algunas keywords similares encontradas:")
            keywords_similares = df_cap_keyword_embedding[
                df_cap_keyword_embedding['keywords'].str.lower().str.contains(
                    keyword_buscada_lower.split()[0] if keyword_buscada_lower.split() else "", 
                    na=False
                )
            ]['keywords'].unique()[:5]  # Mostrar primeras 5 únicas
            
            for kw in keywords_similares:
                print(f"   - '{kw}'")
        
        return None
    
    print(f"\nSe encontraron {len(coincidencias)} keywords con búsqueda {tipo_busqueda} para: '{keyword_buscada}'")
    
    # Procesar resultados
    resultados = []
    for idx, row in coincidencias.head(top_n).iterrows():
        # Extraer información del capítulo
        titulo_completo = row['keywords']
        partes = titulo_completo.split(',', 1)  # Dividir en título y keywords
        
        if len(partes) == 2:
            titulo_capitulo = partes[0].strip()
            keyword_especifica = partes[1].strip()
        else:
            titulo_capitulo = titulo_completo
            keyword_especifica = titulo_completo
        
        capitulo_info = {
            'ID': row.get('id', 'N/A'),
            'Curso': row['curso'],
            'Número_Capítulo': row['numero'],
            'Título_Capítulo': titulo_capitulo,
            'Keyword_Específica': keyword_especifica,
            'Keyword_Completa': titulo_completo,
            'Tipo_Busqueda': tipo_busqueda,
            'Keyword_Buscada': keyword_buscada
        }
        resultados.append(capitulo_info)
    
    # Crear DataFrame de resultados
    df_resultados = pd.DataFrame(resultados)
    
    # Mostrar resultados detallados
    print(f"\n--- RESULTADOS DETALLADOS ({tipo_busqueda.upper()}) PARA '{keyword_buscada.upper()}' ---")
    
    # Agrupar por curso y capítulo para mostrar organizado
    capitulos_unicos = df_resultados[['Curso', 'Número_Capítulo', 'Título_Capítulo']].drop_duplicates()
    
    for idx, row in capitulos_unicos.iterrows():
        print(f"\n🎯 {row['Curso']} - Capítulo {row['Número_Capítulo']}: {row['Título_Capítulo']}")
        
        # Mostrar keywords específicas de este capítulo
        keywords_capitulo = df_resultados[
            (df_resultados['Curso'] == row['Curso']) & 
            (df_resultados['Número_Capítulo'] == row['Número_Capítulo'])
        ]['Keyword_Específica'].tolist()
        
        for i, kw in enumerate(keywords_capitulo, 1):
            print(f"   {i}. {kw}")
    
    # Estadísticas finales
    cursos_afectados = df_resultados['Curso'].nunique()
    capitulos_afectados = df_resultados[['Curso', 'Número_Capítulo']].drop_duplicates().shape[0]
    
    print(f"\n--- ESTADÍSTICAS FINALES ---")
    print(f"📊 Total de keywords encontradas: {len(coincidencias)}")
    print(f"🏫 Cursos afectados: {cursos_afectados}")
    print(f"📚 Capítulos afectados: {capitulos_afectados}")
    print(f"🔍 Tipo de búsqueda: {tipo_busqueda}")
    print(f"🔍 Keyword buscada: '{keyword_buscada}'")
    
    # Guardar en CSV si se solicita
    if guardar_csv:
        # Crear nombre de archivo seguro (sin caracteres especiales)
        keyword_segura = "".join(c for c in keyword_buscada if c.isalnum() or c in (' ', '-', '_')).rstrip()
        keyword_segura = keyword_segura.replace(' ', '_')[:50]  # Limitar longitud
        
        nombre_archivo = f"keyword_{keyword_segura}_capitulos_asociados_{tipo_busqueda}.csv"
        
        try:
            df_resultados.to_csv(nombre_archivo, index=False, encoding='utf-8')
            print(f"💾 Resultados guardados en: {nombre_archivo}")
        except Exception as e:
            print(f"❌ Error al guardar archivo: {e}")
    
    return df_resultados

def aplicacion_2_keywords_exacta_flexible(keyword_buscada, df_cap_keyword_embedding, guardar_csv=False):
    """
    Búsqueda exacta FLEXIBLE - busca la keyword exacta pero dentro de una lista de keywords
    separadas por comas. Es case-insensitive y permite espacios extras.
    """
    print("=" * 70)
    print("APLICACIÓN 2: Búsqueda Exacta Flexible")
    print("(Keyword exacta en lista separada por comas)")
    print("=" * 70)
    
    # Verificar que el DataFrame tiene las columnas necesarias
    columnas_requeridas = ['curso', 'numero', 'keywords']
    for col in columnas_requeridas:
        if col not in df_cap_keyword_embedding.columns:
            print(f"Error: Columna '{col}' no encontrada en el DataFrame")
            return None
    
    keyword_buscada_lower = keyword_buscada.lower().strip()
    
    def contiene_exacta_flexible(texto):
        if pd.isna(texto):
            return False
        
        texto_lower = str(texto).lower()
        
        # Dividir el texto en partes (título y keywords)
        partes = texto_lower.split(',')
        
        # La primera parte es el título del capítulo, el resto son keywords
        if len(partes) > 1:
            # Tomar todas las partes después del título como keywords
            keywords_parte = partes[1:]
            # Unir y luego dividir por comas para obtener keywords individuales
            todas_keywords = ','.join(keywords_parte)
            keywords_individuales = [kw.strip() for kw in todas_keywords.split(',')]
            
            # Buscar la keyword exacta en la lista de keywords
            return keyword_buscada_lower in keywords_individuales
        else:
            # Si no hay comas, buscar en el texto completo
            return keyword_buscada_lower == texto_lower.strip()
    
    mascara = df_cap_keyword_embedding['keywords'].apply(contiene_exacta_flexible)
    coincidencias = df_cap_keyword_embedding[mascara]
    
    if len(coincidencias) == 0:
        print(f"No se encontraron keywords exactas flexibles para: '{keyword_buscada}'")
        
        # Mostrar algunas keywords disponibles para ayudar
        print(f"\n🔍 Keywords disponibles en el DataFrame (primeras 10):")
        keywords_unicas = df_cap_keyword_embedding['keywords'].dropna().unique()[:10]
        for kw in keywords_unicas:
            # Extraer solo las keywords (después de la primera coma)
            if ',' in kw:
                titulo, *keywords = kw.split(',')
                print(f"   - Título: {titulo.strip()}")
                print(f"     Keywords: {[k.strip() for k in keywords]}")
            else:
                print(f"   - {kw}")
        
        return None
    
    print(f"\nSe encontraron {len(coincidencias)} keywords exactas flexibles para: '{keyword_buscada}'")
    
    # Procesar resultados
    resultados = []
    for idx, row in coincidencias.iterrows():
        titulo_completo = row['keywords']
        partes = titulo_completo.split(',', 1)
        
        if len(partes) == 2:
            titulo_capitulo = partes[0].strip()
            keyword_especifica = partes[1].strip()
        else:
            titulo_capitulo = titulo_completo
            keyword_especifica = titulo_completo
        
        capitulo_info = {
            'ID': row.get('id', 'N/A'),
            'Curso': row['curso'],
            'Número_Capítulo': row['numero'],
            'Título_Capítulo': titulo_capitulo,
            'Keyword_Específica': keyword_especifica,
            'Keyword_Completa': titulo_completo,
            'Tipo_Busqueda': 'exacta_flexible',
            'Keyword_Buscada': keyword_buscada
        }
        resultados.append(capitulo_info)
    
    df_resultados = pd.DataFrame(resultados)
    
    # Mostrar resultados
    print(f"\n--- RESULTADOS BÚSQUEDA EXACTA FLEXIBLE ---")
    print(f"Buscando: '{keyword_buscada}' (exactamente en lista de keywords)")
    
    capitulos_unicos = df_resultados[['Curso', 'Número_Capítulo', 'Título_Capítulo']].drop_duplicates()
    
    for idx, row in capitulos_unicos.iterrows():
        print(f"\n🎯 {row['Curso']} - Capítulo {row['Número_Capítulo']}: {row['Título_Capítulo']}")
        
        keywords_capitulo = df_resultados[
            (df_resultados['Curso'] == row['Curso']) & 
            (df_resultados['Número_Capítulo'] == row['Número_Capítulo'])
        ]['Keyword_Específica'].tolist()
        
        # Resaltar la keyword encontrada
        for i, kw in enumerate(keywords_capitulo, 1):
            if keyword_buscada_lower in kw.lower():
                kw_resaltada = kw.replace(
                    keyword_buscada, 
                    f"**{keyword_buscada}**"
                )
                print(f"   {i}. {kw_resaltada} ← ENCONTRADA")
            else:
                print(f"   {i}. {kw}")
    
    # Estadísticas
    cursos_afectados = df_resultados['Curso'].nunique()
    capitulos_afectados = df_resultados[['Curso', 'Número_Capítulo']].drop_duplicates().shape[0]
    
    print(f"\n--- ESTADÍSTICAS ---")
    print(f"📊 Total de keywords encontradas: {len(coincidencias)}")
    print(f"🏫 Cursos afectados: {cursos_afectados}")
    print(f"📚 Capítulos afectados: {capitulos_afectados}")
    print(f"🔍 Tipo de búsqueda: exacta_flexible")
    print(f"🔍 Keyword buscada: '{keyword_buscada}'")
    
    # Guardar si se solicita
    if guardar_csv:
        keyword_segura = "".join(c for c in keyword_buscada if c.isalnum() or c in (' ', '-', '_')).rstrip()
        keyword_segura = keyword_segura.replace(' ', '_')[:50]
        nombre_archivo = f"keyword_{keyword_segura}_capitulos_asociados_exacta_flexible.csv"
        
        try:
            df_resultados.to_csv(nombre_archivo, index=False, encoding='utf-8')
            print(f"💾 Resultados guardados en: {nombre_archivo}")
        except Exception as e:
            print(f"❌ Error al guardar archivo: {e}")
    
    return df_resultados


def aplicacion_2_interactiva(df_cap_keyword_embedding):
    """
    Versión interactiva CORREGIDA que permite múltiples búsquedas con ambas modalidades
    """
    print("🔍 MODO INTERACTIVO - Aplicación 2 (CORREGIDO)")
    print("Opciones de búsqueda:")
    print("  1. Búsqueda que contiene la keyword")
    print("  2. Búsqueda exacta de la keyword (puede fallar si no es idéntica)") 
    print("  3. Búsqueda exacta flexible (c/r a las mayusculas y espacios)")
    print("  4. Comparar todas las búsquedas")
    print("Ingrese 'salir' para terminar\n")
    
    while True:
        keyword = input("Ingrese keyword a buscar: ").strip()
        
        if keyword.lower() == 'salir':
            print("¡Hasta luego!")
            break
            
        if not keyword:
            print("Por favor, ingrese una keyword válida.")
            continue
        
        # Preguntar por modalidad
        print("\nSeleccione modalidad:")
        print("1. Que contiene la keyword")
        print("2. Exacta (coincidencia perfecta)")
        print("3. Exacta flexible (substring exacto)")
        print("4. Comparar todas")
        
        opcion = input("Opción (1/2/3/4): ").strip()
        
        guardar = input("¿Guardar resultados en CSV? (s/n): ").strip().lower() == 's'
        
        if opcion == '1':
            resultados = aplicacion_2_keywords_capitulos(
                keyword, df_cap_keyword_embedding, 
                modo_busqueda="la_contiene", 
                guardar_csv=guardar
            )
        elif opcion == '2':
            resultados = aplicacion_2_keywords_capitulos(
                keyword, df_cap_keyword_embedding, 
                modo_busqueda="exacta", 
                guardar_csv=guardar
            )
        elif opcion == '3':
            resultados = aplicacion_2_keywords_exacta_flexible(
                keyword, df_cap_keyword_embedding, 
                guardar_csv=guardar
            )
        elif opcion == '4':
            print("\n" + "="*50)
            print("COMPARACIÓN DE TODAS LAS BÚSQUEDAS")
            print("="*50)
            
            # Búsqueda que contiene
            print("\n🔍 BÚSQUEDA QUE CONTIENE:")
            resultados_contiene = aplicacion_2_keywords_capitulos(
                keyword, df_cap_keyword_embedding, 
                modo_busqueda="la_contiene", 
                guardar_csv=guardar
            )
            
            print("\n" + "-"*50)
            
            # Búsqueda exacta
            print("\n🔍 BÚSQUEDA EXACTA:")
            resultados_exacta = aplicacion_2_keywords_capitulos(
                keyword, df_cap_keyword_embedding, 
                modo_busqueda="exacta", 
                guardar_csv=guardar
            )
            
            print("\n" + "-"*50)
            
            # Búsqueda exacta flexible
            print("\n🔍 BÚSQUEDA EXACTA FLEXIBLE:")
            resultados_flexible = aplicacion_2_keywords_exacta_flexible(
                keyword, df_cap_keyword_embedding, 
                guardar_csv=guardar
            )
            
            # Comparación
            print("\n📊 COMPARACIÓN FINAL:")
            cont_contiene = len(resultados_contiene) if resultados_contiene is not None else 0
            cont_exacta = len(resultados_exacta) if resultados_exacta is not None else 0
            cont_flexible = len(resultados_flexible) if resultados_flexible is not None else 0
            
            print(f"Búsqueda que contiene: {cont_contiene} resultados")
            print(f"Búsqueda exacta: {cont_exacta} resultados")
            print(f"Búsqueda exacta flexible: {cont_flexible} resultados")
            
            # Recomendación
            if cont_contiene > 0 and cont_exacta == 0:
                print(f"\n💡 Recomendación: Usa 'que contiene' o 'exacta flexible' para '{keyword}'")
        
        else:
            print("Opción no válida. Usando búsqueda que contiene por defecto.")
            resultados = aplicacion_2_keywords_capitulos(
                keyword, df_cap_keyword_embedding, 
                modo_busqueda="la_contiene", 
                guardar_csv=guardar
            )
        
        print("\n" + "="*70 + "\n")

def analisis_comparativo_keywords(keywords_list, df_cap_keyword_embedding, guardar_csv=False):
    """
    Analiza múltiples keywords y muestra comparativas con ambas modalidades
    """
    print("=" * 70)
    print("ANÁLISIS COMPARATIVO DE MÚLTIPLES KEYWORDS")
    print("=" * 70)
    
    resultados_totales = {}
    
    for keyword in keywords_list:
        print(f"\n>>> Analizando: '{keyword}'")
        
        # Procesar ambas modalidades
        resultados_modo = {}
        
        for modo in ["la_contiene", "exacta"]:
            mascara = None
            if modo == "la_contiene":
                mascara = df_cap_keyword_embedding['keywords'].str.lower().str.contains(
                    keyword.lower(), na=False
                )
            else:  # modo exacta
                def contiene_exacta(texto):
                    if pd.isna(texto):
                        return False
                    texto_lower = str(texto).lower()
                    # Búsqueda exacta corregida
                    return keyword.lower() == texto_lower.strip()
                
                mascara = df_cap_keyword_embedding['keywords'].apply(contiene_exacta)
            
            coincidencias = df_cap_keyword_embedding[mascara]
            
            if len(coincidencias) > 0:
                cursos_afectados = coincidencias['curso'].nunique()
                capitulos_afectados = coincidencias[['curso', 'numero']].drop_duplicates().shape[0]
                
                resultados_modo[modo] = {
                    'keywords_encontradas': len(coincidencias),
                    'cursos_afectados': cursos_afectados,
                    'capitulos_afectados': capitulos_afectados,
                    'coincidencias': coincidencias
                }
                
                print(f"   ✅ {modo}: {len(coincidencias)} keywords, {cursos_afectados} cursos, {capitulos_afectados} capítulos")
            else:
                resultados_modo[modo] = None
                print(f"   ❌ {modo}: No se encontraron resultados")
        
        resultados_totales[keyword] = resultados_modo
    
    # Resumen comparativo
    print(f"\n--- RESUMEN COMPARATIVO ---")
    for keyword, modos in resultados_totales.items():
        print(f"\n'{keyword}':")
        for modo, datos in modos.items():
            if datos:
                print(f"  {modo}: {datos['keywords_encontradas']} keywords, {datos['capitulos_afectados']} capítulos")
    
    # Guardar resultados comparativos si se solicita
    if guardar_csv:
        # Crear DataFrame consolidado
        datos_consolidados = []
        for keyword, modos in resultados_totales.items():
            for modo, datos in modos.items():
                if datos:
                    datos_consolidados.append({
                        'Keyword': keyword,
                        'Tipo_Busqueda': modo,
                        'Keywords_Encontradas': datos['keywords_encontradas'],
                        'Cursos_Afectados': datos['cursos_afectados'],
                        'Capitulos_Afectados': datos['capitulos_afectados']
                    })
        
        if datos_consolidados:
            df_comparativo = pd.DataFrame(datos_consolidados)
            nombre_archivo = "comparativo_keywords_modos_busqueda.csv"
            df_comparativo.to_csv(nombre_archivo, index=False, encoding='utf-8')
            print(f"\n💾 Resultados comparativos guardados en: {nombre_archivo}")
    
    return resultados_totales

# Ejemplos de uso rápido
def ejemplos_rapidos(df_cap_keyword_embedding):
    """Ejemplos rápidos para probar ambas modalidades"""
    
    print("🚀 EJEMPLOS RÁPIDOS - Aplicación 2")
    
    # Ejemplo 1: Búsqueda que contiene
    print("\n1. Búsqueda que contiene 'números':")
    aplicacion_2_keywords_capitulos("números", df_cap_keyword_embedding, modo_busqueda="la_contiene", guardar_csv=False)
    
    # Ejemplo 2: Búsqueda exacta  
    print("\n2. Búsqueda exacta de 'suma':")
    aplicacion_2_keywords_capitulos("suma", df_cap_keyword_embedding, modo_busqueda="exacta", guardar_csv=False)
    
    # Ejemplo 3: Búsqueda exacta flexible
    print("\n3. Búsqueda exacta flexible de 'comparar y ordenar':")
    aplicacion_2_keywords_exacta_flexible("comparar y ordenar", df_cap_keyword_embedding, guardar_csv=False)

# =============================================================================
# INSTRUCCIONES DE USO:
# =============================================================================

# Para usar el modo interactivo (RECOMENDADO):
# aplicacion_2_interactiva(df_cap_keyword_embedding)

# Para uso individual:
# resultados = aplicacion_2_keywords_capitulos("números", df_cap_keyword_embedding, modo_busqueda="la_contiene", guardar_csv=True)

# Para búsqueda exacta flexible:
# resultados = aplicacion_2_keywords_exacta_flexible("comparar y ordenar", df_cap_keyword_embedding, guardar_csv=True)

# Para análisis comparativo de múltiples keywords:
# keywords_comparar = ["números", "suma", "resta", "geometría"]
# analisis_comparativo_keywords(keywords_comparar, df_cap_keyword_embedding, guardar_csv=True)

# Para ejemplos rápidos:
#ejemplos_rapidos(df_cap_keyword_embedding)

🚀 EJEMPLOS RÁPIDOS - Aplicación 2

1. Búsqueda que contiene 'números':

2. Búsqueda exacta de 'suma':

3. Búsqueda exacta flexible de 'comparar y ordenar':
APLICACIÓN 2: Búsqueda Exacta Flexible
(Keyword exacta en lista separada por comas)

Se encontraron 2 keywords exactas flexibles para: 'comparar y ordenar'

--- RESULTADOS BÚSQUEDA EXACTA FLEXIBLE ---
Buscando: 'comparar y ordenar' (exactamente en lista de keywords)

🎯 Primero Básico - Capítulo 1: comparar y ordenar
   1. **comparar y ordenar** ← ENCONTRADA

🎯 Tercero Básico - Capítulo 1: comparar y ordenar
   1. **comparar y ordenar** ← ENCONTRADA

--- ESTADÍSTICAS ---
📊 Total de keywords encontradas: 2
🏫 Cursos afectados: 2
📚 Capítulos afectados: 2
🔍 Tipo de búsqueda: exacta_flexible
🔍 Keyword buscada: 'comparar y ordenar'
